In [1]:
from config.paths import BASE_DIR, HF_CACHE_DIR, TRAIN_DIR, VAL_DIR, TEST_DIR

print("Base directory:", BASE_DIR)

✅ Ensured directory exists: /proj/ciptmp/du69limo/to/rag-qa/.hf_cache
✅ Ensured directory exists: /proj/ciptmp/du69limo/to/rag-qa/data
✅ Ensured directory exists: /proj/ciptmp/du69limo/to/rag-qa/data/train
✅ Ensured directory exists: /proj/ciptmp/du69limo/to/rag-qa/data/validation
✅ Ensured directory exists: /proj/ciptmp/du69limo/to/rag-qa/data/test
Base directory: /proj/ciptmp/du69limo/to/rag-qa


In [2]:
# Single cell: safe to run on a fresh environment
from src.load_data import ensure_data_available

# ✅ Creates folders if missing and downloads only if needed
ensure_data_available()

print("🚀 Dataset ready")

/proj/ciptmp/du69limo/to/rag-qa/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✔ Dataset already downloaded — skipping.
🚀 Dataset ready


In [3]:
# Single cell to explore dataset shards
from src.explore_data import load_shards, explore_dataset
from config.paths import TRAIN_DIR, VAL_DIR, TEST_DIR

# Load first few shards to save memory
train_ds = load_shards(TRAIN_DIR, max_shards=3)
val_ds   = load_shards(VAL_DIR, max_shards=3)
test_ds  = load_shards(TEST_DIR, max_shards=3)

# Explore datasets
explore_dataset(train_ds, "Train set")
explore_dataset(val_ds, "Validation set")
explore_dataset(test_ds, "Test set")



Exploring Train set:
Total examples across all shards: 3000
Columns: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer']

Column types:
 - question: Value('string')
 - question_id: Value('string')
 - question_source: Value('string')
 - entity_pages: {'doc_source': List(Value('string')), 'filename': List(Value('string')), 'title': List(Value('string')), 'wiki_context': List(Value('string'))}
 - search_results: {'description': List(Value('null')), 'filename': List(Value('null')), 'rank': List(Value('null')), 'search_context': List(Value('null')), 'title': List(Value('null')), 'url': List(Value('null'))}
 - answer: {'aliases': List(Value('string')), 'matched_wiki_entity_name': Value('string'), 'normalized_aliases': List(Value('string')), 'normalized_matched_wiki_entity_name': Value('string'), 'normalized_value': Value('string'), 'type': Value('string'), 'value': Value('string')}

Sample data from first 3 examples (strings truncated to 50 chars):
{'q

In [4]:
from src.analyze_data import load_shards_concat, dataset_info, analyze_lengths, most_common_answers, print_sample_qa
from config.paths import TRAIN_DIR, VAL_DIR, TEST_DIR

# Load datasets
train_ds = load_shards_concat(TRAIN_DIR)
val_ds   = load_shards_concat(VAL_DIR)
test_ds  = load_shards_concat(TEST_DIR)

# Explore datasets and save plots in the 'plots/' folder
# for name, ds in [("Train", train_ds), ("Validation", val_ds), ("Test", test_ds)]:
#     if ds is None:
#         print(f"No dataset found for {name}")
#         continue
#     dataset_info(ds, name)
#     analyze_lengths(ds, "question", name)
#     analyze_lengths(ds, "answer", name)
#     most_common_answers(ds)
#     print_sample_qa(ds, name, n=5)


In [5]:
from src.compute_embeddings import compute_embeddings

# Compute embeddings or load existing ones
compute_embeddings()


Embeddings not found or force_recompute=True, computing embeddings...


Loading shards: 100%|██████████| 54/54 [00:00<00:00, 163.62it/s]


Loaded dataset with 53988 examples.


Extracting passages: 100%|██████████| 53988/53988 [01:05<00:00, 829.69it/s] 


Removed 0 duplicate passages (100 unique).
Using device: cuda


Computing embeddings: 100%|██████████| 1/1 [00:00<00:00,  3.34it/s]

Saved embeddings to /proj/ciptmp/du69limo/to/rag-qa/src/../corpus_embeddings_unique.pkl


(['Mediterranean Sea: The Mediterranean Sea (pronounced) is a sea connected to the Atlantic Ocean surrounded by the Mediterranean Basin and almost completely enclosed by land: on the north by Southern Europe and Anatolia, on the south by North Africa, and on the east by the Levant. The sea is sometimes considered a part of the Atlantic Ocean, although it is usually identified as a separate body of water. The name Mediterranean is derived from the Latin mediterraneus, meaning "inland" or "in the middle of land" (from medius, "middle" and terra, "land"). It covers an approximate area of 2.5 million km2 (965,000 sq mi), but its connection to the Atlantic (the Strait of Gibraltar) is only 14 km wide. The Strait of Gibraltar is a narrow strait that connects the Atlantic Ocean to the Mediterranean Sea and separates Gibraltar and Spain in Europe from Morocco in Africa. In oceanography, it is sometimes called the Eurafrican Mediterranean Sea or the European Mediterranean Sea to distinguish it 

In [7]:
from src.compute_embeddings import compute_embeddings, retrieve_top_k

# Compute embeddings (will load from file if already exists)
corpus, corpus_embeddings = compute_embeddings()

# Test retrieval
query = "What is the capital of france?"
results, scores = retrieve_top_k(query, corpus, corpus_embeddings, top_k=3)

print("\nTop 3 retrieved passages for query:")
for passage, score in zip(results, scores):
    print(f"[score: {score:.4f}] {passage}\n---")


Loading saved embeddings from /proj/ciptmp/du69limo/to/rag-qa/src/../corpus_embeddings_unique.pkl...
Loaded 100 passages.



Top 3 retrieved passages for query:
[score: 0.3155] Mediterranean Sea: the city of Haifa, Israel File:Gaza Beach.jpg|Beach on the Gaza Strip, State of Palestine File:Coast of Alexandria, A view From Bibliotheca Alexandrina, Egypt.jpg|Coast of Alexandria, view From Bibliotheca Alexandrina, Egypt |A beach in Benghazi, Libya File:ForbysIbizaTown 02.jpg|Old city of Ibiza Town, Spain File:Les Aiguades.jpg|Les Aiguades near Béjaïa, Algeria File:EL Jebha1.jpg|El Jebha, a port town in Morocco File:Gibraltar-Europa-Point-LH-from-the-sea.jpg|Europa Point, Gibraltar File:Monaco City 001.jpg|Panoramic view of La Condamine, Monaco
---
[score: 0.2940] Mediterranean Sea: of Cape Trafalgar (Spain) and Cape Spartel (Africa). **On the northeast: The west coast of Italy. In the Strait of Messina a line joining the north extreme of Cape Paci (15°42'E) with Cape Peloro, the east extreme of the Island of Sicily. The north coast of Sicily. ** On the east: A line joining Cape Lilibeo the western point of Sic